In [9]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3", temperature=0)

In [10]:
system_prompt_template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

What is France's capital?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bonjour! The capital of France is Paris!<|eot_id|><|start_header_id|>user<|end_header_id|>

What can I do there?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Paris, the City of Light, offers a romantic getaway with must-see attractions like the Eiffel Tower and Louvre Museum, romantic experiences like river cruises and charming neighborhoods, and delicious food and drink options, with helpful tips for making the most of your trip.<|eot_id|><|start_header_id|>user<|end_header_id|>

Give me a detailed list of the attractions I should visit, and time it takes in each one, to plan my trip accordingly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""


In [24]:
!pip install langchain
!pip install -qU langchain-openai langchain_chroma
!pip install -qU langchain-text-splitters
!pip install langchainhub

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://repo.kakao.com/pypi/simple/, http://kakaobrain-pypi.dev.9rum.cc/, https://pypi.org/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://repo.kakao.com/pypi/simple/, http://kakaobrain-pypi.dev.9rum.cc/, https://pypi.org/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 14.9 MB/s eta 0:00:00


In [108]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [164]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

loader = WebBaseLoader(
    web_paths=(urls),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [165]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=20,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)
"""
separators=[
        "\n\n",
        "\n",
        ".",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
"""
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-large"), collection_name="asss")

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("agent memory")

/home/bc-user/.local/lib/python3.10/site-packages/langchain_core/vectorstores.py:391: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


'Agent memory refers to the ability of an agent to store information over time and access it when needed. It enables agents to learn from past experiences and make informed decisions in the future. Agent memory is a crucial component in artificial intelligence systems for improving performance and adaptability.'

In [173]:
# 5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 ) - llama3 prompt format 준수
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# prompt로 하면 관련이 있다고 나오네?
joke_query = "agent memory"

parser = JsonOutputParser()
result = rag_chain.invoke("agent memory")
prompt = PromptTemplate(
    template="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant for makes documents and queries relevance.\nDon't repeat what you've already said.\nyou should answer about relevance(yes or no) between query and documents.\n{format_instructions} <|eot_id|><|start_header_id|>user<|end_header_id|>\documents:{documents}\n\nquery:{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
    input_variables=["query", "documents"],
    partial_variables={"format_instructions": parser.get_format_instructions()},

)

chain = ( {"documents": retriever | format_docs, "query": RunnablePassthrough()} | prompt | llm | parser)
#chain = ( prompt | llm | parser)

#chain.invoke({"query": joke_query, "documents": result})
chain.invoke(joke_query)

/home/bc-user/.local/lib/python3.10/site-packages/langchain_core/vectorstores.py:391: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(
/home/bc-user/.local/lib/python3.10/site-packages/langchain_core/vectorstores.py:391: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'relevance': 'yes'}

In [167]:
RunnablePassthrough()

RunnablePassthrough()

In [178]:
# 5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 ) - llama3 prompt format 준수
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# prompt로 하면 관련이 있다고 나오네?
joke_query = "agent memory"
joke_query2 = "I like an apple"
rag_res = rag_chain.invoke(joke_query2)
parser = JsonOutputParser()

prompt = PromptTemplate(
    template="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant for You are the program that determines the hallucination of documents and rag_result. you must answer about hallucination (yes or no) between rag_result and documents.\n{format_instructions} <|eot_id|><|start_header_id|>user<|end_header_id|>\n\n\rag_result:{rag_result}\ndocuments:{documents}<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
    input_variables=["rag_result", "documents"],
    partial_variables={"format_instructions": parser.get_format_instructions()},

)

chain = ( {"documents": retriever | format_docs, "rag_result": RunnablePassthrough()} | prompt | llm | parser)
res = chain.invoke(rag_res)
print(res)
if res['hallucination'].strip() == "no":
    print(retriever.invoke(joke_query))

/home/bc-user/.local/lib/python3.10/site-packages/langchain_core/vectorstores.py:391: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(
/home/bc-user/.local/lib/python3.10/site-packages/langchain_core/vectorstores.py:391: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'hallucination': 'no'}
[]


/home/bc-user/.local/lib/python3.10/site-packages/langchain_core/vectorstores.py:391: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(
